In [1]:
import csv
import json

### Remote I/O

In [2]:
gtfs_partial_shape_file = "../../data/external/gtfs-snippets/airport-routes.csv"
output_line_file = "../../data/interim/airport-routes.geojson"

### Reductions

In [3]:
# Load the CSV file
with open(gtfs_partial_shape_file, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row for row in reader]

# Create a dictionary to store the LineStrings for each shape_id
line_strings = {}

# Iterate over the data and create a LineString for each shape_id
for row in data:
    shape_id = row['shape_id']
    lat = float(row['lat'])
    lon = float(row['lon'])
    shape_sequence = int(row['shape_pt_sequence'])

    # Create a point for this row
    point = [lon, lat]

    # Add the point to the LineString for this shape_id
    if shape_id not in line_strings:
        line_strings[shape_id] = {'type': 'LineString', 'coordinates': []}
    line_strings[shape_id]['coordinates'].append(point)

# Sort the coordinates for each LineString by shape_sequence
for shape_id, line_string in line_strings.items():
    line_string['coordinates'].sort(key=lambda x: data[int(x[1])]['shape_pt_sequence'])

# Create a GeoJSON feature collection
features = []
for shape_id, line_string in line_strings.items():
    feature = {
        'type': 'Feature',
        'geometry': line_string,
        'properties': {'shape_id': shape_id}
    }
    features.append(feature)


### Writes

In [4]:
with open(output_line_file, 'w') as geojson_file:
    json.dump({'type': 'FeatureCollection', 'features': features}, geojson_file)